In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
main_df = pd.read_csv('Food_Inspections_and_Violations.csv')
complaint_df = pd.read_csv('311_Service_Requests_Sanitation_Code_Complaints_Historical.csv')
temperature_df = pd.read_csv('ncei_noaa_govaccesspast_weatherchicago.csv')

In [93]:
main_df.columns

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

Our predicting target should be "Results" but only a subset which includes 'Fail' , 'Pass w/ Conditions' , 'Pass'.    
Since hierarchical relationship exists between above three types (Fail< Pass w/ Conditions < Pass), muticlassification is not ideal.     
Thus regression is being adopted with  Fail=0 , Pass w/ Conditions =1 , and Pass =2

In [94]:
main_df=main_df[main_df["Results"].isin(["Fail","Pass","Pass w/ Conditions"])]
print(main_df["Results"].value_counts())
main_df.head()

Pass                  79053
Fail                  25341
Pass w/ Conditions    13105
Name: Results, dtype: int64


,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,08/28/2017,License,Pass,NaN,41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,08/28/2017,License,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)"
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079.0,CHARTER SCHOOL,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639.0,08/28/2017,License Re-Inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711,"(41.921675488910864, -87.7767113569357)"
5,2079104,SHRIMP'S FISH CHICKEN,SHRIMP'S FISH CHICKEN,2549497.0,Restaurant,Risk 2 (Medium),4355 S COTTAGE GROVE AVE,CHICAGO,IL,60653.0,08/28/2017,License,Fail,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.815197,-87.606531,"(41.81519729173754, -87.60653088909952)"
6,2079091,ART'S DRIVE INN,ART'S DRIVE INN,64682.0,Restaurant,Risk 1 (High),1333 W NORTH AVE,CHICAGO,IL,60622.0,08/25/2017,Canvass,Fail,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.910601,-87.661863,"(41.910600662840906, -87.66186347385191)"


In [95]:
# 79053+25341+13105=117499
print(main_df["Facility Type"].value_counts()[:10])
print(main_df["Facility Type"].value_counts()[:10]/main_df.shape[0])

Restaurant                         79821
Grocery Store                      15825
School                              7998
Bakery                              1780
Daycare (2 - 6 Years)               1685
Children's Services Facility        1480
Daycare Above and Under 2 Years     1353
Long Term Care                       707
Catering                             650
Mobile Food Dispenser                569
Name: Facility Type, dtype: int64
Restaurant                         0.679333
Grocery Store                      0.134682
School                             0.068069
Bakery                             0.015149
Daycare (2 - 6 Years)              0.014341
Children's Services Facility       0.012596
Daycare Above and Under 2 Years    0.011515
Long Term Care                     0.006017
Catering                           0.005532
Mobile Food Dispenser              0.004843
Name: Facility Type, dtype: float64


In [96]:
fac_list=list(main_df["Facility Type"].value_counts()[:7].index)

In [97]:
#only preserve top 7 categories, the rest are grouped into "other"
main_df["Facility Type"]=np.where(main_df["Facility Type"].isin(fac_list),main_df["Facility Type"],"other")

In [98]:
print(main_df["Risk"].value_counts())
main_df=main_df[main_df["Risk"].isin(["Risk 1 (High)","Risk 2 (Medium)","Risk 3 (Low)"])]

Risk 1 (High)      85359
Risk 2 (Medium)    23879
Risk 3 (Low)        8237
All                    5
Name: Risk, dtype: int64


In [99]:
print(main_df["Inspection Type"].value_counts()[:10])
print(main_df["Inspection Type"].value_counts()[:10]/main_df.shape[0])

Canvass                     57911
License                     16183
Canvass Re-Inspection       14253
Complaint                   11561
License Re-Inspection        5599
Complaint Re-Inspection      4916
Short Form Complaint         4679
Suspected Food Poisoning      473
Consultation                  410
License-Task Force            366
Name: Inspection Type, dtype: int64
Canvass                     0.492964
License                     0.137757
Canvass Re-Inspection       0.121328
Complaint                   0.098412
License Re-Inspection       0.047661
Complaint Re-Inspection     0.041847
Short Form Complaint        0.039830
Suspected Food Poisoning    0.004026
Consultation                0.003490
License-Task Force          0.003116
Name: Inspection Type, dtype: float64


In [100]:
ins_list=list(main_df["Inspection Type"].value_counts()[:7].index)
#only preserve top 7 categories, the rest are grouped into "other"
main_df["Inspection Type"]=np.where(main_df["Inspection Type"].isin(ins_list),main_df["Inspection Type"],"other")

In [101]:
main_df["Violations"].head()

0                                                  NaN
3    35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...
4    18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
5    2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...
6    2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...
Name: Violations, dtype: object

In [103]:
print(main_df["Violations"][3])
re.findall(r'\d+\. ',main_df["Violations"][3])

35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: MUST PROVIDE OVERHEAD PROTECTION FOR COFFEE/TEA STATION OUTSIDE OF BALLROOM ON 4TH FL. AND ON 2ND FL. OUTSIDE OF BANQUET KITCHEN IN COORIDOR.  | 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: FLOORS AT COFFEE/TEA STATIONS MUST BE MADE SMOOTH AND EASILY CLEANABLE.


['35. ', '34. ']

To extract the specific number of violations mention for each time, following function will be applied, but note that current violation could not be used as predictor but only previous violation could, otherwise would lead to data leakage.

In [105]:
def extract_violation(x):
    temp_list=[]
    for i in re.findall(r'\d+\. ',x):
        temp_list.append(int(i[:-2]))
    return temp_list


In [106]:
main_df["Violations_bool"]=np.where(main_df["Violations"].isnull(),0,1)
main_df["Violations"].fillna("",inplace=True)
main_df["violations_list"]=main_df["Violations"].apply(lambda x: extract_violation(x))

In [107]:
main_df["violations_len"]=main_df["violations_list"].apply(lambda x: len(x))

In [108]:
main_df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Violations_bool,violations_list,violations_len
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,08/28/2017,License,Pass,,41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)",0,[],0
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,08/28/2017,License,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)",1,"[35, 34]",2
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079.0,other,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639.0,08/28/2017,License Re-Inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711,"(41.921675488910864, -87.7767113569357)",1,"[18, 31, 34, 38, 41]",5
5,2079104,SHRIMP'S FISH CHICKEN,SHRIMP'S FISH CHICKEN,2549497.0,Restaurant,Risk 2 (Medium),4355 S COTTAGE GROVE AVE,CHICAGO,IL,60653.0,08/28/2017,License,Fail,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.815197,-87.606531,"(41.81519729173754, -87.60653088909952)",1,"[2, 11, 21, 22, 30, 32, 33, 34, 35, 37, 38, 41]",12
6,2079091,ART'S DRIVE INN,ART'S DRIVE INN,64682.0,Restaurant,Risk 1 (High),1333 W NORTH AVE,CHICAGO,IL,60622.0,08/25/2017,Canvass,Fail,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.910601,-87.661863,"(41.910600662840906, -87.66186347385191)",1,"[2, 3, 29, 90, 35, 33, 32, 38]",8


In [109]:
violations_dict={}
for i in main_df["violations_list"]:
    for j in i:
        if j in violations_dict:
            violations_dict[j]+=1
        else:
            violations_dict[j]=1

In [110]:
count_df=pd.DataFrame(violations_dict,index=["count"]).T.sort_values(by="count",ascending=False)
count_df["percentage"]=count_df["count"]/count_df["count"].sum()
print(count_df.shape[0])
count_df.head(30)

1007


,count,percentage
34,65240,0.124915
33,57757,0.110587
35,57525,0.110143
38,49551,0.094875
32,48907,0.093642
41,31169,0.059679
18,24502,0.046914
36,23552,0.045095
30,18782,0.035962
40,14853,0.028439


In [111]:
print(main_df[main_df["violations_list"].apply(lambda x: 34 in x)]["Results"].value_counts(normalize=True))
1-main_df[main_df["violations_list"].apply(lambda x: 34 in x)]["Results"].value_counts(normalize=True)[0]

Pass                  0.651759
Fail                  0.233956
Pass w/ Conditions    0.114285
Name: Results, dtype: float64


0.34824146489954577

In [112]:
fail_rate_list=[]
for i in count_df.index[:30]:
    fail_rate_list.append(main_df[main_df["violations_list"].apply(lambda x: i in x)]["Results"].value_counts(normalize=True)[1])
subset_count_df=count_df.head(30)
subset_count_df["fail_rate"]=fail_rate_list

C:\Users\87032\AppData\Local\Temp\ipykernel_5108\857272037.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_count_df["fail_rate"]=fail_rate_list


In [116]:
print(subset_count_df["percentage"].sum())
subset_count_df.sort_values(by="fail_rate",ascending=False)
#18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS
#29. PREVIOUS MINOR VIOLATION(S) CORRECTED 7-42-090
#24. DISH WASHING FACILITIES: PROPERLY DESIGNED, CONSTRUCTED, MAINTAINED, INSTALLED, LOCATED AND OPERATED
#19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE AREA; CLEAN, RODENT PROOF, ALL CONTAINERS COVERED
#3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATURE REQUIREMENT DURING STORAGE, PREPARATION DISPLAY AND SERVICE
#12. HAND WASHING FACILITIES: WITH SOAP AND SANITARY HAND DRYING DEVICES, CONVENIENT AND ACCESSIBLE TO FOOD PREP AREA
#21. * CERTIFIED FOOD MANAGER ON SITE WHEN POTENTIALLY HAZARDOUS FOODS ARE  PREPARED AND SERVED
#16. FOOD PROTECTED DURING STORAGE, PREPARATION, DISPLAY, SERVICE AND TRANSPORTATION

0.9677890000478676


,count,percentage,fail_rate
18,24502,0.046914,0.424267
29,9049,0.017326,0.405110
8,2590,0.004959,0.403407
24,4612,0.008831,0.402865
19,5915,0.011325,0.396239
3,7179,0.013746,0.396134
26,2072,0.003967,0.395360
12,5299,0.010146,0.393137
21,9496,0.018182,0.350785
2,6615,0.012666,0.335822


According to fail_rate could split the violation code to 3 tiers     
tier_0: 18, 29, 8, 24, 19, 3, 26, 12    
tier_1: 21, 2, 11, 16, 9, 14, 37, 30    
tier_2: 41, 36, 40, 38, 35, 32, 43, 45, 34, 31, 33, 42


In [117]:
tier_0 = [18, 29, 8, 24, 19, 3, 26, 12]
tier_1  = [21, 2, 11, 16, 9, 14, 37, 30]    
tier_2 = [41, 36, 40, 38, 35, 32, 43, 45, 34, 31, 33, 42]
main_df["violations_tier_0"]=main_df["violations_list"].apply(lambda x: len(set(x).intersection(set(tier_0))) if len(set(x).intersection(set(tier_0)))>0 else 0)
main_df["violations_tier_1"]=main_df["violations_list"].apply(lambda x: len(set(x).intersection(set(tier_1))) if len(set(x).intersection(set(tier_1)))>0 else 0)
main_df["violations_tier_2"]=main_df["violations_list"].apply(lambda x: len(set(x).intersection(set(tier_2))) if len(set(x).intersection(set(tier_2)))>0 else 0)
#only violations_tier_0, violations_tier_1, violations_tier_2 are used in the model since it embedded the information of violations_len and is more granular


In [119]:
main_df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Violations,Latitude,Longitude,Location,Violations_bool,violations_list,violations_len,violations_tier_0,violations_tier_1,violations_tier_2
0,2079132,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517328.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,...,,41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)",0,[],0,0,0,0
3,2079123,MARRIOT MARQUIS CHICAGO,MARRIOT MARQUIS CHICAGO,2517338.0,Restaurant,Risk 1 (High),2121 S PRAIRIE AVE,CHICAGO,IL,60616.0,...,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.853651,-87.620534,"(41.853650885040594, -87.62053358114167)",1,"[35, 34]",2,0,0,2
4,2079105,CHARTWELLS,CICS WEST BELDEN CAMPUS,2549079.0,other,Risk 1 (High),2245 N MCVICKER AVE,CHICAGO,IL,60639.0,...,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.921675,-87.776711,"(41.921675488910864, -87.7767113569357)",1,"[18, 31, 34, 38, 41]",5,1,0,4
5,2079104,SHRIMP'S FISH CHICKEN,SHRIMP'S FISH CHICKEN,2549497.0,Restaurant,Risk 2 (Medium),4355 S COTTAGE GROVE AVE,CHICAGO,IL,60653.0,...,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.815197,-87.606531,"(41.81519729173754, -87.60653088909952)",1,"[2, 11, 21, 22, 30, 32, 33, 34, 35, 37, 38, 41]",12,0,5,6
6,2079091,ART'S DRIVE INN,ART'S DRIVE INN,64682.0,Restaurant,Risk 1 (High),1333 W NORTH AVE,CHICAGO,IL,60622.0,...,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.910601,-87.661863,"(41.910600662840906, -87.66186347385191)",1,"[2, 3, 29, 90, 35, 33, 32, 38]",8,2,1,4
